In [14]:
import sqlite3
import os
import pandas as pd
import streamlit as st
import time

connexion = sqlite3.connect("../database/bce.db")
cursor = connexion.cursor()

In [3]:
# creating dataframe for entreprise table
cursor.execute("""
Select *
FROM enterprise
""")
enterprise = cursor.fetchall()
col = [description[0] for description in cursor.description]

df_enterprise = pd.DataFrame.from_records(data=enterprise, columns=col)

In [20]:
# creating dataframe for type of enterprise
cursor.execute("""
select * 
from code
where Category = 'TypeOfEnterprise'
""")
data = cursor.fetchall()
col = [description[0] for description in cursor.description]

df_type_of_enterprise = pd.DataFrame.from_records(data=data, columns=col)
df_type_of_enterprise_FR = df_type_of_enterprise.drop(df_type_of_enterprise[df_type_of_enterprise.Language == 'NL'].index)
df_type_of_enterprise_FR

,Category,Code,Language,Description
0,TypeOfEnterprise,0,FR,Inconnu
2,TypeOfEnterprise,1,FR,Personne physique
4,TypeOfEnterprise,2,FR,Personne morale


In [3]:
nbr_of_enterprises = df_enterprise.shape[0]
nbr_of_enterprises

1863292

In [6]:
# Check no duplicates
df_enterprise.duplicated().value_counts()

False    1863292
dtype: int64

In [30]:
df_enterprise.head()

,EnterpriseNumber,Status,JuridicalSituation,TypeOfEnterprise,JuridicalForm,StartDate
0,0200.065.765,AC,000,2,416,1960-08-09 00:00:00
1,0200.068.636,AC,000,2,417,1923-02-16 00:00:00
2,0200.171.970,AC,000,2,116,1968-01-01 00:00:00
3,0200.245.711,AC,012,2,116,1922-01-01 00:00:00
4,0200.305.493,AC,000,2,416,1922-01-01 00:00:00


In [72]:
type(df_enterprise.at[0, 'TypeOfEnterprise'])

str

In [11]:
# How many companies per status
status = df_enterprise['Status'].value_counts()

AC    1863292
Name: Status, dtype: int64

In [4]:
groupby_status = df_enterprise.groupby(['Status']).count()
groupby_status

,EnterpriseNumber,JuridicalSituation,TypeOfEnterprise,JuridicalForm,StartDate
Status,,,,,
AC,1863292,1863292,1863292,1099649,1863291


In [9]:
status_percent = groupby_status['EnterpriseNumber'] / nbr_of_enterprises * 100
status_percent['AC']

100.0

In [18]:
#"Which percentage of the companies are which type of entreprise?"
groupby_type = df_enterprise.groupby(['TypeOfEnterprise']).count()
groupby_type

,EnterpriseNumber,Status,JuridicalSituation,JuridicalForm,StartDate
TypeOfEnterprise,,,,,
1,763643,763643,763643,0,763643
2,1099649,1099649,1099649,1099649,1099648


In [54]:
desc = df_type_of_enterprise_FR.loc[df_type_of_enterprise_FR['Code'] == '2']['Description']
type(desc.values[0])

str

In [4]:
# define query executor function
def run_query(query):
    cursor.execute(query)
    return cursor.fetchall()

In [39]:
start_time = time.time()

rows = run_query('''
    select enterprise.EnterpriseNumber, TypeOfEnterprise.Description
from enterprise
left join (Select Code, Description
	from code
	where code.Category = "TypeOfEnterprise" AND code."Language"="FR")  TypeOfEnterprise
on enterprise.TypeOfEnterprise = TypeOfEnterprise.Code
limit 10000
   ''')

end_time = time.time()
print(f"Query Complete - Elapsed {(end_time - start_time)} secs for 10000 rows")


Query Complete - Elapsed 25.04547142982483 secs for 10000 rows


In [40]:
cols = ['EnterpriseNumber', 'type_desc']
df_from_query = pd.DataFrame(rows, columns=cols)
df_from_query['type_desc'].value_counts()

Personne morale    10000
Name: type_desc, dtype: int64

In [45]:
new_df = pd.merge(df_enterprise, df_from_query, how='left', on="EnterpriseNumber")
new_df.fillna('Personne physique', inplace=True)
new_df

,EnterpriseNumber,Status,JuridicalSituation,TypeOfEnterprise,JuridicalForm,StartDate,type_desc
0,0200.065.765,AC,000,2,416,1960-08-09 00:00:00,Personne morale
1,0200.068.636,AC,000,2,417,1923-02-16 00:00:00,Personne morale
2,0200.171.970,AC,000,2,116,1968-01-01 00:00:00,Personne morale
3,0200.245.711,AC,012,2,116,1922-01-01 00:00:00,Personne morale
4,0200.305.493,AC,000,2,416,1922-01-01 00:00:00,Personne morale
...,...,...,...,...,...,...,...
1863287,0998.603.518,AC,000,2,230,2001-01-01 00:00:00,Personne physique
1863288,0998.603.617,AC,000,2,230,2001-01-01 00:00:00,Personne physique
1863289,0998.604.211,AC,000,2,230,2001-09-15 00:00:00,Personne physique
1863290,0998.604.607,AC,000,2,230,2003-05-07 00:00:00,Personne physique


In [74]:
#def get_type_description(row):
#    if row == '1':
#        print('jqjqj')
#        desc = df_type_of_enterprise_FR.loc[df_type_of_enterprise_FR['Code'] == '1', ['Description']]
#    elif row == '2':
#        desc = df_type_of_enterprise_FR.loc[df_type_of_enterprise_FR['Code'] == '2', ['Description']]
#    else:
#        desc = df_type_of_enterprise_FR.loc[df_type_of_enterprise_FR['Code'] == '0', ['Description']]
#    return desc.values[0] # to get the string value

In [75]:
#df_enterprise['type_description'] = df_enterprise['TypeOfEnterprise'].apply(get_type_description)
#df_enterprise.head(20)

KeyboardInterrupt: 

In [2]:
#cursor.execute("""
#    SELECT JuridicalSituation, count(EnterpriseNumber) as nbr
#from enterprise
#group by JuridicalSituation;
#""")
#var = cursor.fetchall()
#print(var)

[('000', 1755422), ('010', 649), ('011', 3), ('012', 25827), ('013', 3233), ('020', 2493), ('030', 424), ('031', 18), ('040', 7), ('041', 39), ('042', 5), ('043', 43), ('048', 74), ('049', 8), ('050', 74732), ('051', 3), ('053', 1), ('091', 275), ('112', 36)]


In [3]:
#def run_query(query):
#    cursor.execute(query)
#    return cursor.fetchall()
#
#rows = run_query('''
#    SELECT JuridicalSituation, count(EnterpriseNumber) as nbr
#    from enterprise
#    group by JuridicalSituation;
#    ''')
#
## DO NOT USE STREAMLIT WITH JUPYTER NOTEBOOK /!\
## Print results.
#for row in rows:
#    st.write(f"{row[0]} has a :{row[1]}:")

2022-10-25 16:20:27.787 
  command:

    streamlit run /home/lemos/becode/data_viz_SQL/venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
